# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_output = pd.read_csv("output_cities.csv")
cities_output

,Unnamed: 0,Wind,Clouds,Humidity,Country,Data,Temperture,Latitude,Longitude,City
0,0,8.79,27,68,PG,1603324047,84.76,-2.57,150.80,kavieng
1,1,7.25,0,90,MN,1603323924,24.13,45.76,106.27,mandalgovi
2,2,4.00,100,74,NZ,1603324048,66.99,-35.10,173.78,kaeo
3,3,8.01,92,92,EC,1603323986,68.00,-0.74,-90.35,puerto ayora
4,4,17.31,100,89,BR,1603324048,66.13,-30.18,-50.21,cidreira
...,...,...,...,...,...,...,...,...,...,...
574,574,5.39,100,68,ZM,1603324035,75.38,-9.35,28.73,nchelenge
575,575,4.72,73,61,VE,1603323805,84.56,8.12,-63.55,ciudad bolivar
576,576,10.11,32,72,PH,1603324036,83.35,11.29,122.04,tibiao
577,577,16.22,56,95,KZ,1603324036,31.39,50.25,58.43,khromtau


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

locations = cities_output[["Latitude", "Longitude"]]
humidity = cities_output["Humidity"].astype(float)

fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
Best_Weather = cities_output.loc[:,  ["Wind", "Clouds", "Humidity", "Country", "Temperture", "City","Latitude", "Longitude"]]

best_weather = Best_Weather.loc[(Best_Weather["Temperture"]<=80)].loc[(Best_Weather["Humidity"]<=50)].loc[(Best_Weather["Clouds"]<=15)].loc[(Best_Weather["Wind"]<=5)]

best_weather



,Wind,Clouds,Humidity,Country,Temperture,City,Latitude,Longitude
26,3.11,0,32,PK,52.09,zhob,31.34,69.45
46,1.63,2,49,TL,74.62,airai,-8.93,125.41
261,3.91,8,26,ZM,71.55,mongu,-15.25,23.13
351,4.63,0,43,CN,51.93,binzhou,37.37,118.02
567,3.65,0,36,CN,54.63,daokou,35.56,114.51
571,3.36,0,24,SA,62.60,riyadh,24.69,46.72


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_df = best_weather
Hotels = []
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    long = row["Longitude"]
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{long}&locationbias=circle:5000@{lat},{long}&rankby=distance&type=lodging&key={g_key}"
    places = requests.get(url).json()
    hotel_name = places["results"][0]["name"]
    #print(hotel_name)
    Hotels.append(hotel_name)
hotel_df["Hotel Name"] = Hotels

hotel_df
   

,Wind,Clouds,Humidity,Country,Temperture,City,Latitude,Longitude,Hotel Name
26,3.11,0,32,PK,52.09,zhob,31.34,69.45,Cooperative office zhob
46,1.63,2,49,TL,74.62,airai,-8.93,125.41,Juvinal Martin
261,3.91,8,26,ZM,71.55,mongu,-15.25,23.13,West zambia conference guest house
351,4.63,0,43,CN,51.93,binzhou,37.37,118.02,Binzhou Water Conservancy Hotel
567,3.65,0,36,CN,54.63,daokou,35.56,114.51,Qianxiren Hotel
571,3.36,0,24,SA,62.60,riyadh,24.69,46.72,عمارة الشلهوب


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [55]:
# Add marker layer ontop of heat map


gmaps.configure(api_key=g_key)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

locations = cities_output[["Latitude", "Longitude"]]
humidity = cities_output["Humidity"].astype(float)

fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)

marker_locations = hotel_df[['Latitude', 'Longitude']]

fig = gmaps.figure()

markers = gmaps.marker_layer(marker_locations)
                             
fig.add_layer(markers)
fig.add_layer(heat_layer)

fig


# Display figure


Figure(layout=FigureLayout(height='420px'))